
# Deep Learning on DNA Sequences

You will build, train, and evaluate neural networks for taxonomic classification of DNA sequences. Specifically, you will load and process DNA sequences from a FASTA file, exploring sequence data, computing k-mers, and building a neural network for classification. 

---
The notebook is prepared for Pytorch, but if you choose to use TensorFlow you may modify the notebook as you see fit   
**You must complete the sections marked with an asterisc (*) with your own code** 


### 1.1 Reading and Exploring the FASTA Dataset

We'll start by reading a FASTA file and extracting the DNA sequences and taxonomic labels from each entry.

The headers in the file are of the form `>sample_i|taxonomic_class`. We'll parse the file, extract the classes, and explore the data distribution.


In [ ]:
from Bio import SeqIO
import matplotlib.pyplot as plt
from collections import Counter

# Read and parse the FASTA file
fasta_file = "HIV_data/train_data.fasta"  # Replace with your file path
sequences = []
labels = []

for record in SeqIO.parse(fasta_file, "fasta"):
    sequences.append(str(record.seq))
    label = record.description.split('|')[1]  # Extract the taxonomic class
    labels.append(label)

# Plot the distribution of labels
label_counts = Counter(labels)
plt.figure(figsize=(10, 6))
plt.bar(label_counts.keys(), label_counts.values())
plt.xlabel('Taxonomic Class')
plt.ylabel('Frequency')
plt.title('Distribution of Taxonomic Classes')
plt.show()

# Plot the distribution of sequence lengths
sequence_lengths = [len(seq) for seq in sequences]
plt.figure(figsize=(10, 6))
plt.hist(sequence_lengths, bins=20, edgecolor='black')
plt.xlabel('Sequence Length')
plt.ylabel('Frequency')
plt.title('Distribution of Sequence Lengths')
plt.show()



### 1.2* Preprocessing DNA Sequences: Computing K-mer Frequencies

K-mer frequencies can be used as input features for classification models. Let's compute the k-mer frequencies for each DNA sequence.


In [ ]:

from itertools import product

# Function to calculate k-mer frequency
def kmer_frequency(sequence, k=3):
    # Write your own implementation
    pass 
    return   # Normalize frequencies

# Compute k-mer frequencies for each sequence
k = None  # Set k an appropriate value for k

features = [kmer_frequency(seq, k) for seq in sequences]



### 1.3* Defining and Training the Neural Network

With the k-mer frequency features, we'll design a fully connected neural network for taxonomic classification using the architecture provided.

#### Model Architecture: `Net_linear`
I suggest a model with three fully connected layers with ReLU activations and Dropout for regularization.


In [ ]:

import torch.nn as nn
import torch

# Define the Net_linear model
class Net_linear(nn.Module):
    def __init__(self, n_input, n_output):
        super(Net_linear, self).__init__()
        # Define your network
        pass 

    def forward(self, x):
        # Ensure input is flattened if necessary
        pass

# Define input and output dimensions
n_input = len(features[0])  # Based on k-mer feature size
n_output = len(set(labels))  # Number of unique classes

# Initialize the model
model = Net_linear(n_input=n_input, n_output=n_output)
print(model)


#### 1.4* Supervised Training of the model
You may run the training process for as many epochs as you wish, but I suggest training for 100 epochs

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Set up the criterion and optimizer
criterion = None
optimizer = None

# Convert features and labels to tensors and create DataLoader
# Mapping labels to integers
label_map = {label: idx for idx, label in enumerate(set(labels))}
mapped_labels = [label_map[lbl] for lbl in labels]

features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(mapped_labels, dtype=torch.long)

dataset = TensorDataset(features_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Define the Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        # Forward pass
        loss = torch.tensor([0.0])  #placeholder for the loss value

        # Backward pass and optimization
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Note: This model can be tuned further by adjusting batch size, learning rate, or Dropout rates



### 1.5 Evaluating on the Validation Set

After training, it's important to evaluate the model on a separate test set to understand its performance on unseen data.

We'll split our dataset further or use a held-out test set to compute accuracy, plot a confusion matrix, and evaluate other metrics if necessary.


In [ ]:

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd


fasta_file = "HIV_data/val_data.fasta"  # Replace with your file path
sequences = []
ids = []

for record in SeqIO.parse(fasta_file, "fasta"):
    sequences.append(str(record.seq))
    label = record.description[0]  # Extract the tid
    ids.append(label)

val_features = [kmer_frequency(seq, k) for seq in sequences]
val_labels = pd.read_csv("HIV_data/val_taxonomy.csv")["taxonomy"].to_list()
mapped_val_labels = [label_map[lbl] for lbl in val_labels]

val_features = torch.tensor(val_features, dtype=torch.float32)
val_labels_tensor = torch.tensor(mapped_val_labels, dtype=torch.long)

# Create DataLoader for test set
test_dataset = TensorDataset(val_features, val_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluation
model.eval()  # Set model to evaluation mode
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.numpy())
        all_labels.extend(targets.numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")

# Plot confusion matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")


### 1.5 (Optional) Hyperparameter Tuning

Tuning hyperparameters such as the learning rate, number of layers, or dropout rates can significantly impact model performance.

Run a grid search to experiment with different learning rates and dropout probabilities. After each run, we can compare results to find the best configuration. That will make your model more robust and potentially will increase performance in the training set. 


In [ ]:
# Hyperparameter tuning - experimenting with different learning rates and dropout rates


## Part 2: Training a 1D Convolutional Neural Network (CNN)

In this final part, we'll train a 1D CNN to classify DNA sequences. This approach involves:

1. **Setting a Maximum Length**: We'll define a maximum sequence length, then trim or pad all sequences to this length.
2. **One-Hot Encoding DNA Sequences**: Convert sequences to one-hot encoding, so that each base (A, C, G, T) is represented in a way compatible with convolutional layers.
3. **Defining and Training the CNN**: We'll use the CNN architecture provided, with five convolutional layers and four fully connected layers.



### 2.1 Setting a Maximum Sequence Length

Convolutional networks require fixed-length inputs. We'll set a maximum sequence length and either trim longer sequences or pad shorter sequences to this length.


In [ ]:

import numpy as np


# Read and parse the FASTA file
fasta_file = "HIV_data/train_data.fasta"  # Replace with your file path
sequences = []
labels = []

for record in SeqIO.parse(fasta_file, "fasta"):
    sequences.append(str(record.seq))
    label = record.description.split('|')[1]  # Extract the taxonomic class
    labels.append(label)


# Define the maximum length for the sequences
max_len = 9500  # Adjust this as needed based on dataset characteristics

# Trim or pad sequences
def pad_or_trim_sequence(sequence, max_len):
    if len(sequence) > max_len:
        return sequence[:max_len]
    else:
        return sequence + 'N' * (max_len - len(sequence))  # Pad with 'N' for unknowns

# Apply to all sequences
processed_sequences = [pad_or_trim_sequence(seq, max_len) for seq in sequences]
print("Sample processed sequence", processed_sequences[0][:10],'...')



### 2.2 One-Hot Encoding DNA Sequences

To prepare the sequences for input into a 1D CNN, we will convert each base (A, C, G, T) into a one-hot encoding vector.


In [ ]:
# Define one-hot encoding for each base
def one_hot_encode(sequence, max_len):
    encoding_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'N': [0, 0, 0, 0]  # Placeholder for padding/unknowns
    }
    encoded_seq = [encoding_dict[base] for base in sequence]
    return np.array(encoded_seq).T  # Transpose to shape (4, max_len)

# Encode all sequences
encoded_sequences = [one_hot_encode(seq, max_len) for seq in processed_sequences]
encoded_sequences = np.stack(encoded_sequences)  # Stack into a single numpy array

# Convert to PyTorch tensor and add a dimension for CNN input
input_tensor = torch.tensor(encoded_sequences, dtype=torch.float32)  # Shape: (batch_size, 1, 4, max_len)
print("Shape of input tensor:", input_tensor.shape)



### 2.3* Defining and Training the CNN

Define the CNN architecture. Here I suggest using five convolutional layers and four fully connected layers as provided in the comments. Each convolutional layer is followed by a ReLU activation, max pooling, batch normalization, and dropout for regularization. But you are free to use the CNN that works for you.


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, n_input, n_output, w, stride=3):
        super(CNN, self).__init__()
        
        # 5 convolutional layers


        # Fully connected layers



        # Max Pool Layer (used after each convolution)

        
        # Dropout and Batch Normalization



    def forward(self, x):
        # Convolutional layers with batch norm, ReLU, and max pooling

        
        # Flatten the output

        # Fully connected layers
        
        return x

# Define model with input and output dimensions
n_input = None  # number of channels (one-hot encoding) * max_len
n_output = None  # Number of classes

# Initialize the model
model = CNN(n_input=n_input, n_output=n_output, w=None, stride=None)  #The selection of an appropriate kernel size is crucial.
print(model)


#### 2.4* Supervised Training of the model
You may run the training process for as many epochs as you wish, but I suggest training for 100 epochs

In [ ]:
# Define criterion and optimizer
criterion = None
optimizer = None

# Convert labels to integer mapping and set up DataLoader
label_map = {label: idx for idx, label in enumerate(set(labels))}
mapped_labels = [label_map[lbl] for lbl in labels]

labels_tensor = torch.tensor(mapped_labels, dtype=torch.long)
dataset = TensorDataset(input_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for inputs, targets in dataloader:
        # Forward pass
        
        loss = torch.tensor([0.0])
        
        # Backward pass and optimization
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

print("Training complete!")

### 3.4 Evaluation on the validatoin set

Similary to what we did in the previous version, we need to load the data from the validation partition and run the trained model for inference

In [ ]:

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd


fasta_file = "HIV_data/val_data.fasta"  # Replace with your file path
sequences = []
ids = []

for record in SeqIO.parse(fasta_file, "fasta"):
    sequences.append(str(record.seq))
    label = record.description[0]  # Extract the tid
    ids.append(label)

processed_sequences = [pad_or_trim_sequence(seq, max_len) for seq in sequences]


# Encode all sequences
encoded_sequences = [one_hot_encode(seq, max_len) for seq in processed_sequences]
encoded_sequences = np.stack(encoded_sequences)  # Stack into a single numpy array

# Convert to PyTorch tensor and add a dimension for CNN input
val_features = torch.tensor(encoded_sequences, dtype=torch.float32)  # Shape: (batch_size, 4, max_len)
print("Shape of input tensor:", input_tensor.shape)


val_labels = pd.read_csv("HIV_data/val_taxonomy.csv")["taxonomy"].to_list()
mapped_val_labels = [label_map[lbl] for lbl in val_labels]
val_labels_tensor = torch.tensor(mapped_val_labels, dtype=torch.long)

# Create DataLoader for test set
test_dataset = TensorDataset(val_features, val_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluation
model.eval()  # Set model to evaluation mode
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.numpy())
        all_labels.extend(targets.numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")

# Plot confusion matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")


> ### Question:
> Was it easier to learn from engineered features or from raw data? Which architecture would you prefer in this task? Why? 